In [1]:
from textaugment import EDA, Wordnet, Translate

In [2]:
import nltk

In [3]:
import markovify

In [4]:
import pandas as pd

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

In [76]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [36]:
train_file = "code/data/raw_data/assert_ade/train_assert_ade_smm4h_task1.csv"
train_dataset = pd.read_csv(train_file, encoding='utf-8')
train_dataset

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022qlOwrjFkm3LH70ZL,What happened with #MS is that Tysabri assoc w...,assert ade,What happened with #MS is that Tysabri assoc w...,healthy okay
1,SMM4H2022PhAXlFD2YbnkKvEg,"Incredible new discovery today, 2 diclofenac ...",assert ade,"Incredible new discovery today, 2 diclofenac ...",healthy okay
2,SMM4H20228VGGUcA6JYI3mv8H,4:49 I am the child when Addy's nuvaring fails...,assert ade,4:49 I am the child when Addy's nuvaring fails...,healthy okay
3,SMM4H2022PW9c4JNHGZnyLkPd,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",assert ade,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",healthy okay
4,SMM4H2022z1deBvV4Cns8pNOm,@USER_______ @USER___________ @USER_________ n...,assert ade,@USER @USER @USER no amount of imodium will st...,healthy okay
...,...,...,...,...,...
13903,SMM4H202209wNvAFRFaDNMzFB,The finding of an increased risk for a cardiac...,assert ade,The finding of an increased risk for a cardiac...,healthy okay
13904,SMM4H2022gxSFyPApwtAC7OsE,"""'@USER__________: i'm sleepless, you've got m...",assert ade,"""'@USER: i'm sleepless, you've got me wide awa...",adverse event problem
13905,SMM4H20220EQd5qRy7WZRUW7N,@USER___: playing league while on vyvanse is t...,assert ade,@USER: playing league while on vyvanse is too ...,adverse event problem
13906,SMM4H2022Q5YjmFxkRdVtTkwC,“@USER_____: Bananas contain a natural chemica...,assert ade,“@USER: Bananas contain a natural chemical tha...,healthy okay


In [94]:
train_ner_file = "code/data/raw_data/ner_ade/train_ner_ade_smm4h_task2.csv"
train_ner_dataset = pd.read_csv(train_ner_file, encoding='utf-8')
train_ner_dataset

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022qlOwrjFkm3LH70ZL,What happened with #MS is that Tysabri assoc w...,ner ade,What happened with #MS is that Tysabri assoc w...,none
1,SMM4H2022PhAXlFD2YbnkKvEg,"Incredible new discovery today, 2 diclofenac ...",ner ade,"Incredible new discovery today, 2 diclofenac ...",none
2,SMM4H20228VGGUcA6JYI3mv8H,4:49 I am the child when Addy's nuvaring fails...,ner ade,4:49 I am the child when Addy's nuvaring fails...,none
3,SMM4H2022PW9c4JNHGZnyLkPd,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",ner ade,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",none
4,SMM4H2022z1deBvV4Cns8pNOm,@USER_______ @USER___________ @USER_________ n...,ner ade,@USER @USER @USER no amount of imodium will st...,none
...,...,...,...,...,...
13903,SMM4H202209wNvAFRFaDNMzFB,The finding of an increased risk for a cardiac...,ner ade,The finding of an increased risk for a cardiac...,none
13904,SMM4H2022gxSFyPApwtAC7OsE,"""'@USER__________: i'm sleepless, you've got m...",ner ade,"""'@USER: i'm sleepless, you've got me wide awa...",sleepless
13905,SMM4H20220EQd5qRy7WZRUW7N,@USER___: playing league while on vyvanse is t...,ner ade,@USER: playing league while on vyvanse is too ...,stressful
13906,SMM4H2022Q5YjmFxkRdVtTkwC,“@USER_____: Bananas contain a natural chemica...,ner ade,“@USER: Bananas contain a natural chemical tha...,none


# Over-under sampling

## First approach: (Assert)

Take half of noADE tweets (6463 tweets), oversample the 982 ADE tweets.

Then, add the rest of the noADE tweets (6463 tweets), and undersample.

In [44]:
ade_tweets = train_dataset[train_dataset["target_text"] == "adverse event problem"]
num_ade_tweets = len(ade_tweets)
ade_labels = ["adverse event problem" for i in range(num_ade_tweets)]
no_ade_tweets = train_dataset[train_dataset["target_text"] == "healthy okay"]
num_no_ade_tweets = len(no_ade_tweets)
no_ade_labels = ["healthy okay" for i in range(num_no_ade_tweets)]

In [45]:
print("ADE tweets:", num_ade_tweets)
print("noADE tweets:", num_no_ade_tweets)

ADE tweets: 982
noADE tweets: 12926


In [66]:
X = pd.concat([ade_tweets, no_ade_tweets[:num_no_ade_tweets // 2]], ignore_index=True)
y = ade_labels + no_ade_labels[:num_no_ade_tweets // 2]

In [68]:
ros = RandomOverSampler(random_state=25)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [69]:
X_second = pd.concat([X_resampled, no_ade_tweets[num_no_ade_tweets // 2:]], ignore_index=True)
y_second = y_resampled + no_ade_labels[num_no_ade_tweets // 2:]

In [88]:
rus = RandomUnderSampler(random_state=25)
X_final, y_final = rus.fit_resample(X_second, y_second)

In [91]:
X_final

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022XyJLNJX4itmryS7I,i'm sick so i had a difflam lozenge and now my...,assert ade,i'm sick so i had a difflam lozenge and now my...,adverse event problem
1,SMM4H2022anMoAMt49ADycI5q,just had a marathon sleep past 24hrs on new do...,assert ade,just had a marathon sleep past 24hrs on new do...,adverse event problem
2,SMM4H2022yJkbvCqCu3tyu306,i'm so irritable when my vyvanse wears off #fu...,assert ade,i'm so irritable when my vyvanse wears off #fu...,adverse event problem
3,SMM4H2022tihPYZxb9cXtT5eU,so my body was goin threw withdrawl? lol didnt...,assert ade,so my body was goin threw withdrawl? lol didnt...,adverse event problem
4,SMM4H2022XFpkkSfGvy2aZqmg,@USER__________ i hate quetiapine. i was so re...,assert ade,@USER i hate quetiapine. i was so relieved to ...,adverse event problem
...,...,...,...,...,...
12921,SMM4H2022MGfeHP4TB8dAsxnb,that awkward moment when i leave my vyvanse bo...,assert ade,that awkward moment when i leave my vyvanse bo...,healthy okay
12922,SMM4H2022rToS8EB90l03sJ3v,"lamotrigine, though significantly elevated in ...",assert ade,"lamotrigine, though significantly elevated in ...",healthy okay
12923,SMM4H2022xfwA4DUjTKcmflux,Enbrel and the Autoimmune Era - Anne Pollock -...,assert ade,Enbrel and the Autoimmune Era - Anne Pollock -...,healthy okay
12924,SMM4H20225g2OON7XbzDPpAEH,Rivaroxaban 2/2 works well for 100's of 1000s ...,assert ade,Rivaroxaban 2/2 works well for 100's of 1000s ...,healthy okay


In [93]:
X_final.to_csv("code/data/over_under_sample/assert_ade/train_assert_ade_smm4h_task1.csv", index=False)

## First approach: (NER)

Take half of noADE tweets (6463 tweets), oversample the 982 ADE tweets.

Then, add the rest of the noADE tweets (6463 tweets), and undersample.

In [111]:
ade_tweets = train_ner_dataset[train_ner_dataset["target_text"] != "none"]
num_ade_tweets = len(ade_tweets)
ade_labels = ["ade" for i in range(num_ade_tweets)]
no_ade_tweets = train_ner_dataset[train_ner_dataset["target_text"] == "none"]
num_no_ade_tweets = len(no_ade_tweets)
no_ade_labels = list(no_ade_tweets["target_text"])

In [113]:
X = pd.concat([ade_tweets, no_ade_tweets[:num_no_ade_tweets // 2]], ignore_index=True)
y = ade_labels + no_ade_labels[:num_no_ade_tweets // 2]

In [114]:
ros = RandomOverSampler(random_state=25)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [120]:
X_second = pd.concat([X_resampled, no_ade_tweets[num_no_ade_tweets // 2:]], ignore_index=True)
y_second = y_resampled + no_ade_labels[num_no_ade_tweets // 2:]

In [121]:
rus = RandomUnderSampler(random_state=25)
X_final, y_final = rus.fit_resample(X_second, y_second)

In [122]:
X_final

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022XyJLNJX4itmryS7I,i'm sick so i had a difflam lozenge and now my...,ner ade,i'm sick so i had a difflam lozenge and now my...,tongue is numb
1,SMM4H2022anMoAMt49ADycI5q,just had a marathon sleep past 24hrs on new do...,ner ade,just had a marathon sleep past 24hrs on new do...,sleep past 24hrs
2,SMM4H2022yJkbvCqCu3tyu306,i'm so irritable when my vyvanse wears off #fu...,ner ade,i'm so irritable when my vyvanse wears off #fu...,irritable
3,SMM4H2022tihPYZxb9cXtT5eU,so my body was goin threw withdrawl? lol didnt...,ner ade,so my body was goin threw withdrawl? lol didnt...,withdrawl
4,SMM4H2022XFpkkSfGvy2aZqmg,@USER__________ i hate quetiapine. i was so re...,ner ade,@USER i hate quetiapine. i was so relieved to ...,zombie
...,...,...,...,...,...
12921,SMM4H2022MGfeHP4TB8dAsxnb,that awkward moment when i leave my vyvanse bo...,ner ade,that awkward moment when i leave my vyvanse bo...,none
12922,SMM4H2022rToS8EB90l03sJ3v,"lamotrigine, though significantly elevated in ...",ner ade,"lamotrigine, though significantly elevated in ...",none
12923,SMM4H2022xfwA4DUjTKcmflux,Enbrel and the Autoimmune Era - Anne Pollock -...,ner ade,Enbrel and the Autoimmune Era - Anne Pollock -...,none
12924,SMM4H20225g2OON7XbzDPpAEH,Rivaroxaban 2/2 works well for 100's of 1000s ...,ner ade,Rivaroxaban 2/2 works well for 100's of 1000s ...,none


In [124]:
X_final.to_csv("code/data/over_under_sample/ner_ade/train_ner_ade_smm4h_task2.csv", index=False)

# Augmentation: ASSERT ADE

In [9]:
train_dataset["target_text"].value_counts()

healthy okay             12786
adverse event problem      953
Name: target_text, dtype: int64

In [11]:
12786 // 5 - 953

1604

In [6]:
ade_tweets = list(train_dataset[train_dataset["target_text"] == "adverse event problem"]["input_text"])
len(ade_tweets)

953

In [7]:
def augment_helper(dataset, augmenter, aug_type):
  if aug_type == "Wordnet":
    augmented_dataset = [augmenter.augment(sentence) for sentence in dataset]
  elif aug_type == "sr":
    augmented_dataset = [augmenter.synonym_replacement(sentence) for sentence in dataset]
  elif aug_type == "rd":
    augmented_dataset = [augmenter.random_deletion(sentence, p=0.2) for sentence in dataset]
  elif aug_type == "rs":
    augmented_dataset = [augmenter.random_swap(sentence) for sentence in dataset]
  elif aug_type == "ri":  
    augmented_dataset = [augmenter.random_insertion(sentence) for sentence in dataset]
  return augmented_dataset

In [8]:
def augment(n_augmentations, dataset, augmenter, aug_type):
    requested_augmentations = n_augmentations
    augmented_sentences = []
    len_ade = len(dataset)
    while True:
        if n_augmentations > len_ade:
            augmented_sentences += augment_helper(dataset, augmenter, aug_type)
        elif n_augmentations < len_ade and n_augmentations > 0:
            augmented_sentences += augment_helper(dataset[:n_augmentations], augmenter, aug_type)
        else:
            break
        n_augmentations -= len_ade
    if len(set(augmented_sentences)) < requested_augmentations:
        n_left = requested_augmentations - len(set(augmented_sentences))
        augmented_sentences += augment_helper(dataset[-n_left:], augmenter, aug_type)
    augmented_sentences = list(set(augmented_sentences))
    return augmented_sentences

In [9]:
def add_to_corpus(train_corpus, augment_corpus, augment_type, filename):
    new_corpus = train_corpus.copy()
    prefix = train_dataset.iloc[0]["prefix"]
    for i, sentence in enumerate(augment_corpus):
        new_row = {'tweet_id': augment_type + "_augmented_" + str(i), 'tweet': sentence, 'prefix': prefix, 'input_text': sentence, 'target_text': "adverse event problem"}
        new_corpus = new_corpus.append(new_row, ignore_index = True)
    new_corpus.to_csv(filename, index=False)
    return new_corpus

In [14]:
eda_augmenter = EDA()

## Synonym Replacement

In [57]:
n = 1604
sr_dataset = augment(n, ade_tweets, eda_augmenter, "sr")
len(sr_dataset)

1604

In [70]:
add_to_corpus(train_dataset, sr_dataset, "sr", "code/data/sr_augmented_data/assert_ade/train_assert_ade_smm4h_task1.csv")

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022x0kYFv6duKl3xmlO,uncontrolled diabetes? try adding metformin to...,assert ade,uncontrolled diabetes? try adding metformin to...,healthy okay
1,SMM4H202292iWj5N4pVIoizuy,Cc @USER___________ RT @USER_____: Bananas con...,assert ade,Cc @USER RT @USER: Bananas contain a natural c...,healthy okay
2,SMM4H2022xIOYGjjnxWpSYcNX,AstraZeneca lifts sales forecast as Nexium kee...,assert ade,AstraZeneca lifts sales forecast as Nexium kee...,healthy okay
3,SMM4H2022YWko22cQO04JUwtZ,@USER_______ if $aegr reports 275 juxtapid rx ...,assert ade,@USER if $aegr reports 275 juxtapid rx that wo...,healthy okay
4,SMM4H2022msu3S4XwtB5WMOjg,"@USER____ sometimes I get lucky, but baclofen ...",assert ade,"@USER sometimes I get lucky, but baclofen is m...",healthy okay
...,...,...,...,...,...
15338,sr_augmented_1599,@USER i was on cymbalta for over a year. in th...,assert ade,@USER i was on cymbalta for over a year. in th...,adverse event problem
15339,sr_augmented_1600,slept! really slept! i think i can assume from...,assert ade,slept! really slept! i think i can assume from...,adverse event problem
15340,sr_augmented_1601,"In other news, I've got my first eldritch side...",assert ade,"In other news, I've got my first eldritch side...",adverse event problem
15341,sr_augmented_1602,day deuce without nicotine: fuck sleep would b...,assert ade,day deuce without nicotine: fuck sleep would b...,adverse event problem


## Random Insertion

In [71]:
n = 1604
ri_dataset = augment(n, ade_tweets, eda_augmenter, "ri")
len(ri_dataset)

1604

In [76]:
add_to_corpus(train_dataset, ri_dataset, "ri", "code/data/ri_augmented_data/assert_ade/train_assert_ade_smm4h_task1.csv")

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022x0kYFv6duKl3xmlO,uncontrolled diabetes? try adding metformin to...,assert ade,uncontrolled diabetes? try adding metformin to...,healthy okay
1,SMM4H202292iWj5N4pVIoizuy,Cc @USER___________ RT @USER_____: Bananas con...,assert ade,Cc @USER RT @USER: Bananas contain a natural c...,healthy okay
2,SMM4H2022xIOYGjjnxWpSYcNX,AstraZeneca lifts sales forecast as Nexium kee...,assert ade,AstraZeneca lifts sales forecast as Nexium kee...,healthy okay
3,SMM4H2022YWko22cQO04JUwtZ,@USER_______ if $aegr reports 275 juxtapid rx ...,assert ade,@USER if $aegr reports 275 juxtapid rx that wo...,healthy okay
4,SMM4H2022msu3S4XwtB5WMOjg,"@USER____ sometimes I get lucky, but baclofen ...",assert ade,"@USER sometimes I get lucky, but baclofen is m...",healthy okay
...,...,...,...,...,...
15338,ri_augmented_1599,upped my effexor dosage 5 days ago. sleep cycl...,assert ade,upped my effexor dosage 5 days ago. sleep cycl...,adverse event problem
15339,ri_augmented_1600,pt of mine died from cipro rt @USER: @USER if ...,assert ade,pt of mine died from cipro rt @USER: @USER if ...,adverse event problem
15340,ri_augmented_1601,@USER trazodone made me extremely angry and i ...,assert ade,@USER trazodone made me extremely angry and i ...,adverse event problem
15341,ri_augmented_1602,Although my bitching about being fat did look ...,assert ade,Although my bitching about being fat did look ...,adverse event problem


## Wordnet

In [18]:
wordnet_augmenter = Wordnet()

In [11]:
n = 1604
wordnet_dataset = augment(n, ade_tweets, wordnet_augmenter, "Wordnet")
len(wordnet_dataset)

1585

In [13]:
add_to_corpus(train_dataset, wordnet_dataset, "wnet", "code/data/wnet_augmented_data/assert_ade/train_assert_ade_smm4h_task1.csv")

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022x0kYFv6duKl3xmlO,uncontrolled diabetes? try adding metformin to...,assert ade,uncontrolled diabetes? try adding metformin to...,healthy okay
1,SMM4H202292iWj5N4pVIoizuy,Cc @USER___________ RT @USER_____: Bananas con...,assert ade,Cc @USER RT @USER: Bananas contain a natural c...,healthy okay
2,SMM4H2022xIOYGjjnxWpSYcNX,AstraZeneca lifts sales forecast as Nexium kee...,assert ade,AstraZeneca lifts sales forecast as Nexium kee...,healthy okay
3,SMM4H2022YWko22cQO04JUwtZ,@USER_______ if $aegr reports 275 juxtapid rx ...,assert ade,@USER if $aegr reports 275 juxtapid rx that wo...,healthy okay
4,SMM4H2022msu3S4XwtB5WMOjg,"@USER____ sometimes I get lucky, but baclofen ...",assert ade,"@USER sometimes I get lucky, but baclofen is m...",healthy okay
...,...,...,...,...,...
15319,wnet_augmented_1580,"@user i think it's an snri, i'm not even sure....",assert ade,"@user i think it's an snri, i'm not even sure....",adverse event problem
15320,wnet_augmented_1581,@user hopefully i would love to chat or talk b...,assert ade,@user hopefully i would love to chat or talk b...,adverse event problem
15321,wnet_augmented_1582,"you live go with geodon, the meds messed up my...",assert ade,"you live go with geodon, the meds messed up my...",adverse event problem
15322,wnet_augmented_1583,welll..decided to get off paxil. withdrawals e...,assert ade,welll..decided to get off paxil. withdrawals e...,adverse event problem


## Markovify

### Fix empty sentences & smm4h false tweets

In [20]:
markov_model = markovify.Text(ade_tweets)

In [34]:
average_length = sum([len(s.split()) for s in ade_tweets]) / len(ade_tweets)
average_length

18.42707240293809

In [37]:
n = 1604
markov_dataset = []
for i in range(n):
    markov_dataset.append(markov_model.make_sentence(min_words=10, max_words=25))

In [41]:
len(set(markov_dataset))

1507

In [42]:
for i in range(n - len(set(markov_dataset))):
    markov_dataset.append(markov_model.make_sentence(min_words=10, max_words=25))

In [43]:
len(set(markov_dataset))

1593

In [39]:
markov_dataset[:10]

['Oh, what a great few hours on my own arm!',
 "@USER lamictal makes you put on w/seroquel, and i've been having them today too, three days without fluoxetine and losartan... so dizzy!",
 'would like less vyvanse and sometimes i have been forgetting everything lately #paxil',
 'little did i just wanted to let you know i gained weight in the past 48 hours.',
 '@USER i hav a terrible rash because of it! had the sane experience of disturbed sleep is brought to you by vyvanse.',
 '@USER @USER @USER my doctor was terrified of me doing it but i take way too strong',
 "@USER natural don't work for me is i don't sleep. if have and have a nightmare.......",
 "@USER Anybody know anything about side effects when i breathe, i'm fine.",
 "humira + thioguanine + preds = me have no crazy stories each morning now i'm sleepy",
 'i seem to be reminded to eat because i ran out of their nicotine addiction.']

In [44]:
add_to_corpus(train_dataset, set(markov_dataset), "markov", "code/data/markov_augmented_data/assert_ade/train_assert_ade_smm4h_task1.csv")

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022x0kYFv6duKl3xmlO,uncontrolled diabetes? try adding metformin to...,assert ade,uncontrolled diabetes? try adding metformin to...,healthy okay
1,SMM4H202292iWj5N4pVIoizuy,Cc @USER___________ RT @USER_____: Bananas con...,assert ade,Cc @USER RT @USER: Bananas contain a natural c...,healthy okay
2,SMM4H2022xIOYGjjnxWpSYcNX,AstraZeneca lifts sales forecast as Nexium kee...,assert ade,AstraZeneca lifts sales forecast as Nexium kee...,healthy okay
3,SMM4H2022YWko22cQO04JUwtZ,@USER_______ if $aegr reports 275 juxtapid rx ...,assert ade,@USER if $aegr reports 275 juxtapid rx that wo...,healthy okay
4,SMM4H2022msu3S4XwtB5WMOjg,"@USER____ sometimes I get lucky, but baclofen ...",assert ade,"@USER sometimes I get lucky, but baclofen is m...",healthy okay
...,...,...,...,...,...
15327,markov_augmented_1588,Numbness and tingling in my nerves and all the...,assert ade,Numbness and tingling in my nerves and all the...,adverse event problem
15328,markov_augmented_1589,ooooop. anyone know if amoxicillin and pregaba...,assert ade,ooooop. anyone know if amoxicillin and pregaba...,adverse event problem
15329,markov_augmented_1590,just took seroquel. now i'm never forgetting m...,assert ade,just took seroquel. now i'm never forgetting m...,adverse event problem
15330,markov_augmented_1591,@USER hi idk who you are but i don't sleep wel...,assert ade,@USER hi idk who you are but i don't sleep wel...,adverse event problem


# Augmentation: NER ADE

In [135]:
ade_train_ner_tweets = train_ner_dataset[train_ner_dataset["target_text"] != "none"]
ade_train_ner_tweets

,tweet_id,tweet,prefix,input_text,target_text
29,SMM4H2022XyJLNJX4itmryS7I,i'm sick so i had a difflam lozenge and now my...,ner ade,i'm sick so i had a difflam lozenge and now my...,tongue is numb
46,SMM4H2022anMoAMt49ADycI5q,just had a marathon sleep past 24hrs on new do...,ner ade,just had a marathon sleep past 24hrs on new do...,sleep past 24hrs
47,SMM4H2022yJkbvCqCu3tyu306,i'm so irritable when my vyvanse wears off #fu...,ner ade,i'm so irritable when my vyvanse wears off #fu...,irritable
49,SMM4H2022tihPYZxb9cXtT5eU,so my body was goin threw withdrawl? lol didnt...,ner ade,so my body was goin threw withdrawl? lol didnt...,withdrawl
50,SMM4H2022XFpkkSfGvy2aZqmg,@USER__________ i hate quetiapine. i was so re...,ner ade,@USER i hate quetiapine. i was so relieved to ...,zombie
...,...,...,...,...,...
13862,SMM4H2022u3SlhKzH2rS0K4xu,@USER____ And then I had horrible sleep once I...,ner ade,@USER And then I had horrible sleep once I too...,horrible sleep
13865,SMM4H2022MaoNl62q5dfg0X43,"""@USER____ natural don't work for me, try 300m...",ner ade,"""@USER natural don't work for me, try 300mg of...",sleep for days
13870,SMM4H2022H46enMioMkbcxr0A,note to self: do not take #fluoxetine and #ale...,ner ade,note to self: do not take #fluoxetine and #ale...,space cadet
13904,SMM4H2022gxSFyPApwtAC7OsE,"""'@USER__________: i'm sleepless, you've got m...",ner ade,"""'@USER: i'm sleepless, you've got me wide awa...",sleepless


In [137]:
noade_train_ner_tweets = train_ner_dataset[train_ner_dataset["target_text"] == "none"]
noade_train_ner_tweets

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022qlOwrjFkm3LH70ZL,What happened with #MS is that Tysabri assoc w...,ner ade,What happened with #MS is that Tysabri assoc w...,none
1,SMM4H2022PhAXlFD2YbnkKvEg,"Incredible new discovery today, 2 diclofenac ...",ner ade,"Incredible new discovery today, 2 diclofenac ...",none
2,SMM4H20228VGGUcA6JYI3mv8H,4:49 I am the child when Addy's nuvaring fails...,ner ade,4:49 I am the child when Addy's nuvaring fails...,none
3,SMM4H2022PW9c4JNHGZnyLkPd,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",ner ade,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",none
4,SMM4H2022z1deBvV4Cns8pNOm,@USER_______ @USER___________ @USER_________ n...,ner ade,@USER @USER @USER no amount of imodium will st...,none
...,...,...,...,...,...
13901,SMM4H2022VuapAHQyP2dZKkTu,effects of Xarelto & Warfarin include exanguin...,ner ade,effects of Xarelto & Warfarin include exanguin...,none
13902,SMM4H2022Ewc6T48zzG8jVBJ7,slutty fluoxetine,ner ade,slutty fluoxetine,none
13903,SMM4H202209wNvAFRFaDNMzFB,The finding of an increased risk for a cardiac...,ner ade,The finding of an increased risk for a cardiac...,none
13906,SMM4H2022Q5YjmFxkRdVtTkwC,“@USER_____: Bananas contain a natural chemica...,ner ade,“@USER: Bananas contain a natural chemical tha...,none


In [150]:
n = 1604
count = 0
augmented_dataset = train_ner_dataset.copy()
while count < n:
    for i, row in ade_train_ner_tweets.iterrows():
        tweet = row["input_text"].lower()
        span = row["target_text"].lower()
        tweet = tweet.replace(span, "@@span@@")
        new_tweet = wordnet_augmenter.augment(tweet)
        if tweet.strip() != new_tweet.strip():
            new_tweet = new_tweet.replace("@@span@@", span)
            if new_tweet not in list(ade_train_ner_tweets["input_text"]):
                row["input_text"] = new_tweet
                row["tweet"] = new_tweet
                row["target_text"] = span
                row["tweet_id"] = "wnet_augmented_" + str(count)
                augmented_dataset = augmented_dataset.append(row, ignore_index = True)
                count += 1
        if count > n:
            break

In [151]:
augmented_dataset

,tweet_id,tweet,prefix,input_text,target_text
0,SMM4H2022qlOwrjFkm3LH70ZL,What happened with #MS is that Tysabri assoc w...,ner ade,What happened with #MS is that Tysabri assoc w...,none
1,SMM4H2022PhAXlFD2YbnkKvEg,"Incredible new discovery today, 2 diclofenac ...",ner ade,"Incredible new discovery today, 2 diclofenac ...",none
2,SMM4H20228VGGUcA6JYI3mv8H,4:49 I am the child when Addy's nuvaring fails...,ner ade,4:49 I am the child when Addy's nuvaring fails...,none
3,SMM4H2022PW9c4JNHGZnyLkPd,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",ner ade,"""zyrtec, benadryl, lortab, etodolac, flexeril,...",none
4,SMM4H2022z1deBvV4Cns8pNOm,@USER_______ @USER___________ @USER_________ n...,ner ade,@USER @USER @USER no amount of imodium will st...,none
...,...,...,...,...,...
15508,wnet_augmented_1600,i'd gladly substitute the #mtx headache for #h...,ner ade,i'd gladly substitute the #mtx headache for #h...,headache
15509,wnet_augmented_1601,"@user right, some ssris live habituate for cat...",ner ade,"@user right, some ssris live habituate for cat...",rebound
15510,wnet_augmented_1602,#effexor #withdrawl subsist a monster that i t...,ner ade,#effexor #withdrawl subsist a monster that i t...,withdrawl
15511,wnet_augmented_1603,i've live so tired since i countenance the cym...,ner ade,i've live so tired since i countenance the cym...,tired


In [152]:
augmented_dataset.to_csv("code/data/wnet_augmented_data/ner_ade/train_ner_ade_smm4h_task2.csv", index=False)